# Analysis Flow

This notebook demonstrates how an Automate Flow can be used to perform analysis. 
We use a flow that combines Globus Transfer and funcX to move data and then execute an analysis code on the data.

In [16]:
from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

import time
import json
import sys
import os

from funcx.sdk.client import FuncXClient

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

## Create an analysis function

Use funcX to register a simple analysis function.

In [17]:
# fxc = FuncXClient()
fxc = FuncXClient(use_offprocess_checker=False)

In [18]:
import funcx
funcx.__version__

'0.0.5'

In [19]:
def file_size(data):
    """Return the size of a file"""
    import os
    return os.path.getsize(data['pathname'])

func_uuid = fxc.register_function(file_size)
print(func_uuid)

b7787e3b-1b78-4579-a772-27b6e80690f6


In [20]:
payload = {'pathname': '/etc/hostname'}
tutorial_endpoint = 'a694a32d-8707-4884-b205-3cfc87c4f667' # Public tutorial endpoint
res = fxc.run(payload, endpoint_id=tutorial_endpoint, function_id=func_uuid)
print(res)

c09de957-2338-4dd5-ae9e-293c7a4ee9fc


In [21]:
fxc.get_result(res)

16

## Create the flow

In [35]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "funcx",
  "States": {
    "funcx": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 600,
      "End": True
    }
  }
}


Register the flow

In [30]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Stills process workflow")
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/e4bf76e9-eb9c-4315-8f98-bb542786b15d', 'administered_by': [], 'api_version': '1.0', 'created_at': '2021-06-03T18:00:42.799114+00:00', 'created_by': 'urn:globus:auth:identity:95278182-10a1-11e6-9c7e-7b385f033313', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'funcx', 'States': {'funcx': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'End': True, 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 600}}}, 'description': '', 'flow_url': 'https://flows.globus.org/flows/e4bf76e9-eb9c-4315-8f98-bb542786b15d', 'globus_auth_scope': 'https://auth.globus.org/scopes/e4bf76e9-eb9c-4315-8f98-bb542786b15d/flow_e4bf76e9_eb9c_4315_8f98_bb542786b15d_user', 'globus_auth_usernam

In [31]:
flow_input = {
    "input": {
        "fx_id": func_uuid,
        "fx_ep": tutorial_endpoint,
        "pathname": '/etc/hostname'
    }
}

Run the flow and wait for it to complete

Note: This will try to transfer a file called "test.txt" from your Globus Tutorial Endpoint 1.

In [32]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
# print(flow_action)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(5)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

Please log into Globus here:
----------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=e6c75d97-532a-4c88-b031-8584a319fa3e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Fe4bf76e9-eb9c-4315-8f98-bb542786b15d%2Fflow_e4bf76e9_eb9c_4315_8f98_bb542786b15d_user&state=_default&response_type=code&code_challenge=rz7cE1c3BMW1QthEX6AFfSkmU58zP72XtLMJJevzRpY&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Command+Line+Interface+on+ripchip
----------------------------

Enter the resulting Authorization Code here: Flow action started with id: eebd6cc7-af94-4575-adb7-5f7582c25652
Flow status: ACTIVE
Flow status: SUCCEEDED


Check the result of the analysis function

In [34]:
flow_action['details']

{'output': {'AnalyzeResult': {'action_id': '1d1ba124-2a2b-4e4c-95b4-1c9abcdf2ea5',
   'details': {'completion_t': '1622743272.9936237',
    'exception': None,
    'result': 16,
    'status': 'SUCCEEDED',
    'task_id': '1d1ba124-2a2b-4e4c-95b4-1c9abcdf2ea5'},
   'release_after': 'P30D',
   'state_name': 'funcx',
   'status': 'SUCCEEDED'},
  'input': {'fx_ep': 'a694a32d-8707-4884-b205-3cfc87c4f667',
   'fx_id': 'b7787e3b-1b78-4579-a772-27b6e80690f6',
   'pathname': '/etc/hostname'}}}